# Terminal Processing

© Crown-owned copyright 2025, Defence Science and Technology Laboratory UK

This notebook serves as a guide on the functionality and use of the `terminal` service from both the simulation and game layers.

In [ ]:
!primaite setup

In [ ]:
from primaite.simulator.system.services.terminal.terminal import Terminal
from primaite.simulator.network.container import Network
from primaite.simulator.network.hardware.nodes.host.computer import Computer
from primaite.simulator.system.applications.red_applications.ransomware_script import RansomwareScript
from primaite.simulator.system.services.terminal.terminal import RemoteTerminalConnection


## Simulation Layer Implementation.

The `terminal` service comes pre-installed on most node types. 

_The only exception to this being `switches` network nodes, this is because PrimAITE currently only implements 'dumb' switches. `routers` and `firewalls` however all support the `terminal`._

In [ ]:
!primaite setup

In this notebook, the terminal is demoed on a basic network consisting of two computers, connected together via a link to form a basic LAN network which can be seen by the `basic_network()` method defined below.

In [ ]:
def basic_network() -> Network:
    """Utility function for creating a default network to demonstrate Terminal functionality"""
    network = Network()
    node_a = Computer.from_config(
            config = {
            "type": "computer",
            "hostname": "node_a",
            "ip_address": "192.168.0.10",
            "subnet_mask": "255.255.255.0",
            # "startup_duration": 0,
        }
    )
    node_a.power_on()
    node_b = Computer.from_config(
        config = {
            "type": "computer",
            "hostname": "node_b",
            "ip_address": "192.168.0.11",
            "subnet_mask": "255.255.255.0",
            # "startup_duration": 0,
        }
    )
    node_b.power_on()
    network.connect(node_a.network_interface[1], node_b.network_interface[1])
    return network

After setting up the network, the terminal can be accessed from a `Node` via the `software_manager`:

In [ ]:
network: Network = basic_network()
computer_a: Computer = network.get_node_by_hostname("node_a")
terminal_a: Terminal = computer_a.software_manager.software.get("terminal")
computer_b: Computer = network.get_node_by_hostname("node_b")
terminal_b: Terminal = computer_b.software_manager.software.get("terminal")

However, before we're able to send commands from `node_a` to `node_b`, you will need to `login` to `node_b` first, using valid user credentials. 

After providing successful credentials, the login method will return type of `TerminalClientConnection` object which can then be used for sending commands to the node. 

In the example below, we are remotely logging in to the default ***'admin'*** account on `node_b`, from `node_a` (If you are not logged in, any commands sent will be rejected by the remote).


In [ ]:
# Login to the remote (node_b) from local (node_a)
term_a_term_b_remote_connection: RemoteTerminalConnection = terminal_a.login(username="admin", password="admin", ip_address="192.168.0.11")

You can view all active connections to a terminal through use of the `show()` method.

In [ ]:
terminal_b.show()

As we logged into a remote node, the login method return a `RemoteTerminalConnection` which allows us to forward commands to be executed on the target node. 

The example below demonstrates how you can remotely install an application on the target node.

In [ ]:
term_a_term_b_remote_connection.execute(["software_manager", "application", "install", "ransomware-script"])

In [ ]:
computer_b.software_manager.show()

As the terminal allows us to leverage the [request system](./Requests-and-Responses.ipynb) we have full access to the request manager on any simulation component. For example, the code snippet below demonstrates how we the `terminal` allows the user of `terminal_a`, on `computer_a`, to send a command (in the form of a request) to `computer_b` to create a downloads folder. 


In [ ]:
# Display the current state of the file system on computer_b
computer_b.file_system.show()

# Send command
term_a_term_b_remote_connection.execute(["file_system", "create", "folder", "downloads"])

The resultant call to `computer_b.file_system.show()` shows that the new folder has been created.

In [ ]:
computer_b.file_system.show()

Information about the latest response when executing a remote command can be seen by calling the `last_response` attribute within `Terminal`

In [ ]:
print(terminal_a.last_response)

When finished, the connection can be closed by calling the `disconnect` function of the Remote Client object

In [ ]:
# Display active connection
terminal_a.show()
terminal_b.show()

term_a_term_b_remote_connection.disconnect()

terminal_a.show()
terminal_b.show()

Disconnected Terminal sessions will no longer show in the node's Terminal connection list, but will be under the historic sessions in the `user_session_manager`.

In [ ]:
computer_b.user_session_manager.show(include_historic=True, include_session_id=True)

## Game Layer Implementation

This notebook section will detail the implementation of how the game layer utilises the terminal to support different agent actions.

The ``terminal`` is directly leveraged to implement the following agent actions.


|  Game Layer Action                | Simulation Layer         |
|-----------------------------------|--------------------------|
| ``node-send-local-command``       | Uses the given user credentials, creates a ``LocalTerminalSession`` and executes the given command and returns the ``RequestResponse``.
| ``node-session-remote-login``                 | Uses the given user credentials and remote IP to create a ``RemoteTerminalSession``.
| ``node-send-remote-command``      | Uses the given remote IP to locate the correct ``RemoteTerminalSession``, executes the given command and returns the ``RequestsResponse``.

Additionally, the `terminal` is utilised extensively by the [c2 suite](./Command-and-Control-E2E-Demonstration.ipynb) and it's related actions. 

### Game Layer Setup

Similar to other notebooks, the next code cells create a custom proxy agent to demonstrate how these commands can be leveraged by agents in the ``UC2`` network environment.

If you're unfamiliar with ``UC2`` then please refer to the [UC2-E2E-Demo notebook for further reference](./Data-Manipulation-E2E-Demonstration.ipynb).

In [ ]:
import yaml
from primaite.config.load import data_manipulation_config_path
from primaite.session.environment import PrimaiteGymEnv

In [ ]:
custom_terminal_agent = """
  - ref: CustomTerminalAgent
    team: RED
    type: proxy-agent
    action_space:
      action_map:
        0:
          action: do-nothing
          options: {}
        1:
          action: node-send-local-command
          options:
            node_name: client_1
            username: admin
            password: admin
            command:
                - file_system
                - create
                - file
                - downloads
                - dog.png
                - False
        2:
          action: node-session-remote-login
          options:
            node_name: client_1
            username: admin
            password: admin
            remote_ip: 192.168.10.22
        3:
          action: node-send-remote-command
          options:
            node_name: client_1
            remote_ip: 192.168.10.22
            command:
                - file_system
                - create
                - file
                - downloads
                - cat.png
                - False
"""
custom_terminal_agent_yaml = yaml.safe_load(custom_terminal_agent)

In [ ]:
with open(data_manipulation_config_path()) as f:
    cfg = yaml.safe_load(f)
    # removing all agents & adding the custom agent.
    cfg['agents'] = {}
    cfg['agents'] = custom_terminal_agent_yaml

env = PrimaiteGymEnv(env_config=cfg)

client_1: Computer = env.game.simulation.network.get_node_by_hostname("client_1")
client_2: Computer = env.game.simulation.network.get_node_by_hostname("client_2")

### Terminal Action | ``node-send-local-command`` 

The yaml snippet below shows all the relevant agent options for this action:

```yaml
      action_map:
        1:
          action: node-send-local-command
          options:
            node_name: client_1
            username: admin
            password: admin
            command:
                - file_system
                - create
                - file
                - downloads
                - dog.png
                - False
```

In [ ]:
env.step(1)
client_1.file_system.show(full=True)

### Terminal Action | ``node-session-remote-login``  

The yaml snippet below shows all the relevant agent options for this action:

```yaml
      action_map:
        2:
          action: node-session-remote-login
          options:
            node_name: client_1
            username: admin
            password: admin
            remote_ip: 192.168.10.22 # client_2's ip address.
```

In [ ]:
env.step(2)
client_2.session_manager.show()

### Terminal Action |  ``node-send-remote-command``

The yaml snippet below shows all the relevant agent options for this action:

```yaml
      action_map:
        3:
          action: node-send-remote-command
          options:
            node_name: client_1
            remote_ip: 192.168.10.22 # client_2's ip address.
            command:
                - file_system
                - create
                - file
                - downloads
                - cat.png
                - False
```

In [ ]:
env.step(3)
client_2.file_system.show(full=True)